In [ ]:
import argparse
import pickle
from tqdm import tqdm
import sys
import pandas as pd
import numpy as np
import os

sys.path.extend(['../'])


training_subjects = [1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 38, 40, 41, 42, 43]
num_joint = 28
max_frame = 50

import json

def read_skeleton_filter(file):
    # Read JSON file
    with open(file, 'r') as f:
        data = json.load(f)

    # Calculate number of frames and joints per frame
    num_frames = len(data['annotations'])
    num_joints = data['annotations'][0]['num_keypoints']

    skeleton_sequence = {
        'numFrame': num_frames,
        'frameInfo': []
    }

    # Iterate over each frame
    for annotation in data['annotations']:
        frame_info = {
            'numJoint': num_joints,
            'jointInfo': []
        }

        keypoints = annotation['keypoints']
        for i in range(0, len(keypoints), 3):
            x = keypoints[i]
            y = keypoints[i + 1]

            joint_info = {
                'x': x,
                'y': y
            }
            frame_info['jointInfo'].append(joint_info)

        skeleton_sequence['frameInfo'].append(frame_info)

    return skeleton_sequence

def read_xy(file, num_joint=28):
    seq_info = read_skeleton_filter(file)
    data = np.zeros((seq_info['numFrame'], num_joint, 2))

    for n, f in enumerate(seq_info['frameInfo']):
        for j, v in enumerate(f['jointInfo']):
            data[n, j, :] = [v['x'], v['y']]  # Include only X and Y coordinates

    data = data.transpose(2, 0, 1)  # Transpose for desired output format (X,Y first)
    return data

In [ ]:
import numpy as np
import os
from tqdm import tqdm
import pickle

max_frame = 50  # Number of frames
num_joint = 28  # Number of joints

def gendata(data_path, out_path, benchmark='xsub', set_name='val'):
    sample_name = []
    sample_label = []

    # Identify sample names and labels
    for filename in os.listdir(data_path):
        print(f"Processing file: {filename}")
        action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
        subject_id = int(filename[filename.find('P') + 1:filename.find('P') + 4])

        # Determine if the sample belongs to training or validation set
        istraining = (subject_id in training_subjects)
        issample = (istraining if set_name == 'train' else not istraining)

        if issample:
            sample_name.append(filename)
            sample_label.append(action_class - 1)  # Convert to 0-based index



    # Prepare to store the chunks and corresponding labels
    fp_list = []
    labels_list = []

    # Process each sample
    for i, s in enumerate(tqdm(sample_name)):
        data = read_xy(os.path.join(data_path, s), num_joint=num_joint)
        num_frames = data.shape[1]
        print(f"Processing file: {s}, data shape: {data.shape}")

        # Create chunks of size 50 frames
        for start_frame in range(0, num_frames, max_frame):
            end_frame = start_frame + max_frame

            # Extract chunk of 50 frames or less
            if end_frame <= num_frames:
                data_chunk = data[:, start_frame:end_frame, :]


            fp_list.append(data_chunk)
            labels_list.append(sample_label[i])

    # Convert the list of chunks to NumPy arrays
    fp = np.array(fp_list)
    labels = np.array(labels_list)

    print(f"Final dataset shape: {fp.shape}")
    print(f"Final labels shape: {labels.shape}")

    # Save the processed data and labels
    np.save(f'{out_path}/{set_name}_data_joint_pad.npy', fp)
    with open(f'{out_path}/{set_name}_labels_pad.pkl', 'wb') as f:
        pickle.dump(labels, f)

if __name__ == '__main__':
    benchmark = ['xsub']
    set_name = ['train', 'val']
    data_path = r"/content/drive/MyDrive/PFE/Model/Horse_Gait_Recognition/Data/Raw"
    out_folder = r'/content/drive/MyDrive/PFE/Model/Horse_Gait_Recognition/Data/Extracted/xsub"

    for b in benchmark:
        for sn in set_name:
            out_path = os.path.join(out_folder, b)
            if not os.path.exists(out_path):
                os.makedirs(out_path)
            print(b, sn)
            gendata(data_path, out_path, benchmark=b, set_name=sn)


xsub train
Processing file: S001P001A001.json
Processing file: S002P002A001.json
Processing file: S003P003A001.json
Processing file: S004P004A001.json
Processing file: S005P005A001.json
Processing file: S006P006A001.json
Processing file: S007P007A001.json
Processing file: S008P008A001.json
Processing file: S009P009A002.json
Processing file: S010P010A002.json
Processing file: S011P011A002.json
Processing file: S012P012A002.json
Processing file: S013P013A003.json
Processing file: S014P014A003.json
Processing file: S015P015A003.json
Processing file: S016P016A003.json
Processing file: S017P017A003.json
Processing file: S018P018A003.json
Processing file: S019P019A001.json
Processing file: S020P020A001.json
Processing file: S021P021A003.json
Processing file: S022P022A001.json
Processing file: S023P023A003.json
Processing file: S024P024A001.json
Processing file: S025P025A003.json
Processing file: S026P026A002.json
Processing file: S027P027A002.json
Processing file: S028P028A001.json
Processin

 41%|████      | 14/34 [00:00<00:00, 130.76it/s]

Processing file: S001P001A001.json, data shape: (2, 197, 28)
Processing file: S002P002A001.json, data shape: (2, 129, 28)
Processing file: S003P003A001.json, data shape: (2, 168, 28)
Processing file: S004P004A001.json, data shape: (2, 102, 28)
Processing file: S006P006A001.json, data shape: (2, 181, 28)
Processing file: S007P007A001.json, data shape: (2, 123, 28)
Processing file: S008P008A001.json, data shape: (2, 180, 28)
Processing file: S010P010A002.json, data shape: (2, 187, 28)
Processing file: S011P011A002.json, data shape: (2, 182, 28)
Processing file: S012P012A002.json, data shape: (2, 191, 28)
Processing file: S013P013A003.json, data shape: (2, 218, 28)
Processing file: S014P014A003.json, data shape: (2, 126, 28)
Processing file: S015P015A003.json, data shape: (2, 211, 28)
Processing file: S016P016A003.json, data shape: (2, 182, 28)
Processing file: S018P018A003.json, data shape: (2, 210, 28)
Processing file: S019P019A001.json, data shape: (2, 200, 28)
Processing file: S020P02

100%|██████████| 34/34 [00:00<00:00, 114.42it/s]


Processing file: S032P032A002.json, data shape: (2, 158, 28)
Processing file: S033P033A003.json, data shape: (2, 114, 28)
Processing file: S034P034A002.json, data shape: (2, 185, 28)
Processing file: S035P035A002.json, data shape: (2, 183, 28)
Processing file: S036P036A001.json, data shape: (2, 182, 28)
Processing file: S038P038A003.json, data shape: (2, 189, 28)
Processing file: S040P040A002.json, data shape: (2, 199, 28)
Processing file: S041P041A003.json, data shape: (2, 151, 28)
Processing file: S042P042A003.json, data shape: (2, 151, 28)
Processing file: S043P043A003.json, data shape: (2, 151, 28)
Final dataset shape: (137, 2, 50, 28)
Final labels shape: (137,)
xsub val
Processing file: S001P001A001.json
Processing file: S002P002A001.json
Processing file: S003P003A001.json
Processing file: S004P004A001.json
Processing file: S005P005A001.json
Processing file: S006P006A001.json
Processing file: S007P007A001.json
Processing file: S008P008A001.json
Processing file: S009P009A002.json
P

100%|██████████| 9/9 [00:00<00:00, 141.03it/s]

Processing file: S005P005A001.json, data shape: (2, 109, 28)
Processing file: S009P009A002.json, data shape: (2, 136, 28)
Processing file: S017P017A003.json, data shape: (2, 91, 28)
Processing file: S023P023A003.json, data shape: (2, 189, 28)
Processing file: S024P024A001.json, data shape: (2, 204, 28)
Processing file: S025P025A003.json, data shape: (2, 188, 28)
Processing file: S030P030A001.json, data shape: (2, 188, 28)
Processing file: S037P037A002.json, data shape: (2, 191, 28)
Processing file: S039P039A002.json, data shape: (2, 208, 28)
Final dataset shape: (34, 2, 50, 28)
Final labels shape: (34,)


In [ ]:

import torch
import math
import numpy as np
import argparse
import os
import torch.nn as nn


# Define directory path for your  data
data_dir = r'C:\Users\hp\OneDrive - Efrei\AD-and-DLB-Classifier-master Baseline+gap\AD-and-DLB-Classifier-master\data\Multiple1\xsub'

# Define filenames for training and testing data (assuming .npy format)
file_train = 'train_data_joint_pad.npy'
file_test = 'val_data_joint_pad.npy'

print('loading data')
X_train = np.load(os.path.join(data_dir,file_train),  allow_pickle=True)
X_test = np.load(os.path.join(data_dir,file_test), allow_pickle=True)
y_train = np.load(os.path.join(data_dir,'train_labels_pad.pkl'), allow_pickle=True)
y_test = np.load(os.path.join(data_dir,'val_labels_pad.pkl'), allow_pickle=True)

y_train = y_train[1]

y_test = y_test[1]

y_train = np.array(y_train).astype('int32')
y_test = np.array(y_test).astype('int32')

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

X_train = X_train.transpose(0, 2 ,3 , 1)
print(X_train.shape)
X_test= X_test.transpose(0, 2 ,3 , 1)
print(X_test.shape)

loading data
(137, 2, 50, 28)
(34, 2, 50, 28)
()
()
(137, 50, 28, 2)
(34, 50, 28, 2)


In [ ]:
import pickle

def load_data_labels(data_dir, set_name):

    with open(os.path.join(data_dir, f'{set_name}_labels_pad.pkl'), 'rb') as f:
        labels = pickle.load(f)
    return  labels

# Example usage
y_train = load_data_labels(data_dir, 'train')
y_test = load_data_labels(data_dir, 'val')

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(137, 50, 28, 2)
(34, 50, 28, 2)
(137,)
(34,)
